<a href="https://colab.research.google.com/github/seyonechithrananda/bert-loves-chemistry/blob/master/SMILES_Tokenizer_PubChem10M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Feb 24 22:44:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install transformers
!pip install wandb

     |████████████████████████████████| 1.9MB 15.9MB/s 
     |████████████████████████████████| 890kB 46.7MB/s 
     |████████████████████████████████| 3.2MB 50.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=25b31a194a70903e9790c31018e3409191fe989164c52bcd4a2c12c9d5e9a77e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 2.0MB 18.9MB/s 
     |████████████████████████████████| 133kB 38.8MB/s 
     |████████████████████████████████| 163kB 52.8MB/s 
     |████████████████████████████████| 102kB 14.3MB/s 
     |████████████████████████████████| 71kB 12.4MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=993412badbad170e32fb489ea5a5c05d8bcd6b93d70d2d53776ba12b518f7061
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968

In [20]:
!pip install nlp

     |████████████████████████████████| 1.7MB 8.5MB/s 
     |████████████████████████████████| 20.7MB 12.9MB/s 
     |████████████████████████████████| 245kB 54.4MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [3]:
import transformers

In [4]:
import torch

In [5]:
import wandb

In [6]:
wandb.login()

wandb: Currently logged in as: seyonec (use `wandb login --relogin` to force relogin)


True

In [7]:
#verify file length

fname = 'pubchem-10m.txt'
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

print(file_len(fname))


10000000


In [8]:
%%time
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer


CPU times: user 13 µs, sys: 3 µs, total: 16 µs
Wall time: 18.4 µs


In [9]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


In [11]:
import torch
torch.cuda.is_available() #checking if CUDA + Colab GPU works


True

In [12]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=512,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("seyonec/SMILES_tokenized_PubChem_shard00_160k", max_len=512)

# test
tokenizer.encode("[O-][N+](=O)c1cnc(s1)Sc1nnc(s1)N")

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


[12,
 19,
 31,
 23,
 17,
 22,
 19,
 18,
 15,
 20,
 15,
 25,
 15,
 17,
 42,
 20,
 18,
 34,
 15,
 20,
 25,
 25,
 15,
 17,
 42,
 20,
 18,
 23,
 13]

In [13]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)
model.num_parameters()

83502880

In [15]:
from torch.utils.data import Dataset
from nlp import load_dataset


class RawTextDataset(Dataset):
    """
    Custom Torch Dataset for tokenizing large (up to 100,000,000+ sequences) text corpuses,
    by not loading the entire dataset into cache and using lazy loading from disk (using huggingface's
    'NLP' library. See 'https://github.com/huggingface/nlp' for more details on the NLP package.
    Examples
    --------
    >>> from raw_text_dataset import RawTextDataset
    >>> dataset = SmilesRawTextDataset(tokenizer=tokenizer, file_path="shard_00_selfies.txt", block_size=512)
    Downloading: 100%
    1.52k/1.52k [00:03<00:00, 447B/s]
    Using custom data configuration default
    Downloading and preparing dataset text/default-f719ef2eb3ab586b (download: Unknown size, generated: Unknown size, post-processed: Unknown sizetotal: Unknown size) to /root/.cache/huggingface/datasets/text/default-f719ef2eb3ab586b/0.0.0/3a79870d85f1982d6a2af884fde86a71c771747b4b161fd302d28ad22adf985b...
    Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-f719ef2eb3ab586b/0.0.0/3a79870d85f1982d6a2af884fde86a71c771747b4b161fd302d28ad22adf985b. Subsequent calls will reuse this data.
    Loaded Dataset
    Number of lines: 999988
    Block size: 512
    """
    def __init__(self, tokenizer, file_path: str, block_size: int):
        self.tokenizer = tokenizer
        self.file_path = file_path
        self.block_size = block_size

        self.dataset = load_dataset("text", data_files=file_path)["train"]
        print("Loaded Dataset")
        self.len = len(self.dataset)
        print("Number of lines: " + str(self.len))
        print("Block size: " + str(self.block_size))

    def __len__(self):
        return self.len

    def preprocess(self, text):
        batch_encoding = self.tokenizer(str(text), add_special_tokens=True, truncation=True, max_length=self.block_size)
        return torch.tensor(batch_encoding["input_ids"])

    def __getitem__(self, i):
        line = self.dataset[i]
        example = self.preprocess(line)
        return example


In [16]:
%%time
dataset = RawTextDataset(tokenizer=tokenizer, file_path="pubchem-10m.txt", block_size=512)

Using custom data configuration default


Loaded Dataset
Number of lines: 10000000
Block size: 512
CPU times: user 121 ms, sys: 270 ms, total: 391 ms
Wall time: 13.6 s


In [17]:
print(dataset[0])
# CN(c1ccccc1)c1ccccc1C(=O)NCC1(O)CCOCC1
tokenizer(str('CN(c1ccccc1)c1ccccc1C(=O)NCC1(O)CCOCC1'), add_special_tokens=True, truncation=True, max_length=128)

tensor([12, 16, 23, 17, 15, 20, 15, 15, 15, 15, 15, 20, 18, 15, 20, 15, 15, 15,
        15, 15, 20, 16, 17, 22, 19, 18, 23, 16, 16, 20, 17, 19, 18, 16, 16, 19,
        16, 16, 20, 13])


{'input_ids': [12, 16, 23, 17, 15, 20, 15, 15, 15, 15, 15, 20, 18, 15, 20, 15, 15, 15, 15, 15, 20, 16, 17, 22, 19, 18, 23, 16, 16, 20, 17, 19, 18, 16, 16, 19, 16, 16, 20, 13], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [18]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [20]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="PubChem_10M_SMILES_Tokenizer",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    fp16 = True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [21]:
from ipywidgets import IntProgress


In [22]:
%%time
trainer.train()


Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss


RuntimeError: ignored

In [23]:
trainer.save_model("PubChem_10M_SMILES_Tokenizer")

# Methods for preventing RuntimeError

- reduce per_gpu_train_batch_size to 16
- reduce seq_length to 128 or below (recommended to be 128 minimum just in case)